In [1]:
from selenium import webdriver
import time
import pandas as pd
import os

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

url1 = 'https://www.linkedin.com/jobs/search/?currentJobId=3724442233&distance=25.0&f_E=2&geoId=103121230&keywords=data%20science%20and%20analytics&origin=JOB_SEARCH_PAGE_JOB_FILTER'

driver = webdriver.Chrome()
driver.get(url1)
driver.implicitly_wait(10)

In [2]:
#Logging in to LinkedIn using a dummy account

signin_button = driver.find_element(By.CLASS_NAME, 'nav__button-secondary')
signin_button.click()
username = driver.find_element(By.ID, 'username')
username.send_keys("erinlovesprogramming@gmail.com")
password = driver.find_element(By.ID, 'password')
password.send_keys("ILoveDev21-")
login_button = driver.find_element(By.CLASS_NAME, 'btn__primary--large')
login_button.click()

In [3]:
#verification code

code = driver.find_element(By.CLASS_NAME, 'form__input--text')
code.send_keys('790344') #edit sccordingly
submit_button = driver.find_element(By.ID, 'email-pin-submit-button')
submit_button.click()

In [3]:
#scraping proper

y = driver.find_elements(By.CLASS_NAME, 'jobs-search-results-list__subtitle')[0].text

try:
    if y.endswith('+') and "," in y:
        cleaned_y = y[:-1].replace(",", "")

    else:
        cleaned_y = y
        
    n = pd.to_numeric(cleaned_y)
    
except:
    parts = y.split()

    n = int(parts[0])

n 

327

In [ ]:
#Loop to scroll through all jobs and click on see more jobs button for infinite scrolling (in logged in accounts, pagination was used)

i = 2
while i <= int((n+400)/25)+1: 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    
    try:
        send = driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']")
        driver.execute_script("arguments[0].click();", send)   
        time.sleep(3)
            
    except:
        pass
        time.sleep(5)
        
companyname = []
titlename = []
joblink = []

#Find company name and append it to the blank list
try:
    for i in range(n):
        company=driver.find_elements(By.CLASS_NAME, 'base-search-card__subtitle')[i].text
        companyname.append(company)
    
except IndexError:
    print('_')
    
len(companyname)

#Find title name and append it to the blank list
try:
    for i in range(n):
        title = driver.find_elements(By.CLASS_NAME, 'base-search-card__title')[i].text
        titlename.append(title)
        
except IndexError:
    print('_')
    
len(titlename)

#Find job link and append it to the blank list
try:
    link = driver.find_elements(By.CLASS_NAME, 'base-card__full-link')
    for k in link:
        joblink.append(k.get_attribute('href'))
        
except IndexError:
    print('_')
    
len(joblink)

In [7]:
#Find skills needed from each job link and append to blank list

skillsneeded = []
try:
    for url2 in joblink:
        driver.get(url2)
        driver.implicitly_wait(10)
        skills = driver.find_elements(By.CLASS_NAME, '') #edit accordingly
        skillsneeded.append(detail)
        
except IndexError:
    print('_')
    
len(jobdetails)

321

In [6]:
#Using Pandas to Create a Table
company_final = pd.DataFrame(companyname, columns = ["Company"])
title_final = pd.DataFrame(titlename, columns = ["Title"])
#links_list = pd.DataFrame(joblink., columns = ["Job Links"])

scraped_jobs = company_final.join(title_final)

scraped_jobs

,Company,Title
0,Nestlé,Data Scientist
1,Grab,Data Scientist (Analytics) - Country & Marketi...
2,Deep Knowledge Group,Data Scientist
3,AffinityX,Data Scientist
4,Megaworld Corporation,Data Analyst
...,...,...
334,Accenture in the Philippines,Microsoft Azure Analytics Services_Cebu
335,Accenture in the Philippines,Data Architect_ATCP
336,Accenture in the Philippines,SAP BusinessObjects Business Intelligence Cons...
337,Accenture in the Philippines,EVERGREEN_DEVOPS_PHL_ATCP


In [ ]:
#Saving to a file in my directory
scraped_jobs.to_csv('datasci_jobs.csv')